In [4]:
# baseado em - https://www.kaggle.com/leandrodoze/sentiment-analysis-in-portuguese
#imports
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict



In [7]:
dataset = pd.read_csv('./files test/Tweets_Mg.csv')
dataset.count()

Unnamed: 0                   8199
Created At                   8199
Text                         8199
Geo Coordinates.latitude      104
Geo Coordinates.longitude     104
User Location                5489
Username                     8199
User Screen Name             8199
Retweet Count                8199
Classificacao                8199
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [8]:
dataset.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Construindo o modelo

In [10]:
tweets = dataset["Text"].values
tweets

array(['���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ',
       '� @ Governador Valadares, Minas Gerais https://t.co/B3ThIDJCSf',
       '�� @ Governador Valadares, Minas Gerais https://t.co/dPkgzVR2Qw',
       ...,
       'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia https://t.co/zaQbXRRJWc',
       'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia: Um dos autores teria molestado vítima de… https://t.co/lQ8cTSNftA',
       'Trio suspeito de roubo de cargas é preso em Santa Luzia (MG) https://t.co/0INgJcMtZb #R7MG #RecordTVMinas'],
      dtype=object)

In [11]:
classes = dataset["Classificacao"].values
classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

In [12]:
# Agora, vamos treinar o modelo usando a abordagem Bag of Words e o algoritmo Naive Bayes Multinomial
#    - Bag of Words, na prática, cria um vetor com cada uma das palavras do texto completo da base,
#      depois, calcula a frequência em que essas palavras ocorrem em uma data sentença, para então
#      classificar/treinar o modelo
#    - Exemplo HIPOTÉTICO de três sentenças vetorizadas "por palavra" e classificadas baseada na
#      frequência de suas palavras:
#         {0,3,2,0,0,1,0,0,0,1, Positivo}
#         {0,0,1,0,0,1,0,1,0,0, Negativo}
#         {0,1,1,0,0,1,0,0,0,0, Neutro}
#    - Olhando para esses vetores, meu palpite é que as palavras nas posições 2 e 3 são as com maior
#      peso na determinação de a que classe pertence cada uma das três sentenças avaliadas
#    - A função fit_transform faz exatamente esse processo: ajusta o modelo, aprende o vocabulário,
#      e transforma os dados de treinamento em feature vectors, a.k.a. vetor com frequêcia das palavras

vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB()

## Testando o Modelo

In [31]:


# Vamos usar algumas frases de teste para fazer a classificação com o modelo treinado
testes = ["Bom dia rapaziada"]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['Neutro'], dtype='<U8')

In [33]:
# Validação cruzada do modelo. Neste caso, o modelo é dividido em 10 partes, treinado em 9 e testado em 1
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
# Quão acurada é a média do modelo?
metrics.accuracy_score(classes, resultados)


0.8831564824978656

## Testando com outra base de dados

In [43]:
df = pd.read_csv('./files test/tweets_para_teste.csv')
df.head()

,Unnamed: 0,ID,USUARIO,TWEET,DATA_TWEET,language
0,6,556,Thayna18900373,RT @lohlovesot7: olha eu digo sem medo de erra...,2020-08-25 12:23:15,portuguese
1,7,557,naturegyu,RT @Tdetravesti: MAIS DE 12 MIL MÁSCARAS!!! \n...,2020-08-25 12:22:59,portuguese
2,9,559,nathaliaspss,RT @TatiNefertari: A Casa Nem é um espaço de a...,2020-08-25 12:21:21,portuguese
3,10,560,ivansearom,RT @pabllovittar: #CASANEMCASAVIVA não dá pra...,2020-08-25 12:16:56,portuguese
4,11,562,lonelybkwolf,RT @pabllovittar: #CASANEMCASAVIVA não dá pra...,2020-08-25 12:15:43,portuguese


In [41]:
def classifica_tweets(tweet):
    tweet = [tweet]
    freq_testes = vectorizer.transform(tweet)
    return modelo.predict(freq_testes)

In [49]:
df_tweets = list(df['TWEET'])
for i in range(100,150):
    predict = classifica_tweets(df_tweets[i])
    print(f"{predict} - {df_tweets[i]}\n")

['Negativo'] - RT @TiaDoZap5: São  todos bundões mesmo!!! Jornalista  que mente, distorce,  esconde,  inventa,  nega a verdade  nem caráter  tem. Chama -l…

['Neutro'] - RT @marcelloneves72: Por tudo que a imprensa faz há 601 dias com o Presidente Bolsonaro, chama-los  de BUNDÕES, ele está sendo muito genero…

['Neutro'] - RT @z_furtado1: O @LulaOficial e a @dilmabr sempre trataram todos os jornalistas com muita atenção e carinho.Foram barbaramente injustiçado…

['Neutro'] - RT @BeccaJB38: Enquanto estamos dando porrada na globo e nos blogueiros bundões o Maia e o Alcolumbre estão articulando com o STF a reeleiç…

['Neutro'] - RT @reinaldoazevedo: Na fala de @jairbolsonaro , o vírus vira seu aliado contra “jornalistas bundoes”.  Art. 268 do Código Penal pune quem…

['Neutro'] - RT @syllavol6: O NOME DELE ERA JACOB BLACK! UMA PETIÇÃO ESTÁ ROLANDO SOBRE ESSE CASO (PETIÇÃO PARA ACUSAR OS POLICIAIS) ASSINEM E COMPARTIL…

['Neutro'] - RT @misforminho: felix de cropped, seungmin quase me mat

## ingnore daqui pra baixo

In [ ]:
def clean()

In [63]:
#df = df.drop(columns = ['DATA_TWEET'])
df.to_csv('./files test/tweets_classificados.csv')
#df['Classificacao'] = '-'
